# Optimizing Model Parameters 
Now that we have a model and data its time to train, validate and test our model by optimizing its parameters on our data. Training a model is an iterative process; in each iteration the model makes a guess about the output, calculates the error in its geuss, coolects the derivatives of the error with respect to its parameters, and optiized these parameters using gradient descent. 
## Prerequisite Code 
We will lode the code from an earlier turotial 

In [18]:
import torch 
from torch import nn
from torch.utils.data import DataLoader 
from torchvision import datasets 
from torchvision.transforms import ToTensor, Lambda 

training_data = datasets.FashionMNIST(
root = 'data',
train = True,
download = True,
transform = ToTensor()
)
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28,512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10),
        nn.ReLU()
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork()

## Hyperparameters
Hyperparameters are adjustable parameters that let you control the model optimization process. Different hyperparmeter values can impact model trainging and convergence rates. We define the follwoing hyperparameters for training: Number of Epochs: The number of times to iterate over the dataset. Batch Size: The number of data samples propagate through the network before the parameters are updated. Learning Rate: how much to update models parameters at each batch/epoch. Smaller values yield slow learning speed, while learge values may result in inpredicatable behaviour durining trainging 

In [19]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## Optimization Loop:
Once we set out hyperparameters, we can then train and optimize out model with an optimization loop. Each iteration of the optimization loops is called an epoch. Each epoch consists of two main points. The train loop: Iterate over the training dataset and tryo to converge to optimal parameters. The validation/Test loop: iterate over the test dataset to check if model performance is improving 
## Loss Function:
When presented with some trainging data, our untrained network is likely not to give the correct answer. The Loss function measures the degree of dissimilairty of obtained reaults to the target value, and it is the loss function that we want to minmize during trainging. To calculate the loss we make a prediction using the inputs of our given data sample compare it against the true data label value. Common loss functions include nn.MSELoss for regression tasks and nn.NLLLoss for classification nn.CrossEntropy combines nn.LogSoftmax and nn.NLLLoss We pass our models output logistics to nn.CrossEntropyLoss, which will normalize the logits and compute the prediction error. 

In [20]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

## Optimizer 
Optimization is the process of adjusting model parameters to reduce model error in each training step. Optimization algorithms define how this process is perfromed. All optimization logic is encapsulated in the optimizer object. Here we used the SGD optimizer; additionally, there are many different optimizers avaliable in PyTorch such as ADAM and RMSProp, that work better for different kinds of models and data. We initalize the optimizer by registering the model's parameters that need to be trained, and passing in the learning rate hyperparameter. 


In [21]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Inside the trainging loop, optimization happens in three steps. Call optimizer.zero_grad() to reset the gradients of model parameters. Gradients by default add up, to prevent double counting, we explicitly zero them at each iteration. Backpropagation the prediction loss with a call to loss.backwards(). Pytorch deposits the gradients of the loss with respect to each paraeter. Once we have our gradients, we call optimizer.step() to adjust the parameters by the gradients collected in the backward pass 

## Full Implementation
We define train_loop that loops over our optimization code, and test_loop that evaluates the models performance against out test data. 


In [24]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
def test_loop(dataloader, model,loss_fn):
    size = len(dataloader.dataset)
    test_loss,correct=0, 0
    
    with torch.no_grad():
        for X,y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= size
    correct /= size 
    print(f"Test Error: \n Accuracy:{(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

We initialize the loss function and optimizer, and pass it to train_loop and test_loop. 

In [25]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

epochs = 10 

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.221707  [    0/60000]
loss: 2.222266  [ 6400/60000]
loss: 2.212206  [12800/60000]
loss: 2.217342  [19200/60000]
loss: 2.117703  [25600/60000]
loss: 2.141768  [32000/60000]
loss: 2.112190  [38400/60000]
loss: 2.085189  [44800/60000]
loss: 2.113314  [51200/60000]
loss: 2.034552  [57600/60000]
Test Error: 
 Accuracy:44.8%, Avg loss: 0.032567 

Epoch 2
-------------------------------
loss: 2.106107  [    0/60000]
loss: 2.113096  [ 6400/60000]
loss: 2.083368  [12800/60000]
loss: 2.086576  [19200/60000]
loss: 1.908256  [25600/60000]
loss: 1.934270  [32000/60000]
loss: 1.923558  [38400/60000]
loss: 1.874394  [44800/60000]
loss: 1.942645  [51200/60000]
loss: 1.787222  [57600/60000]
Test Error: 
 Accuracy:47.1%, Avg loss: 0.029609 

Epoch 3
-------------------------------
loss: 1.957973  [    0/60000]
loss: 1.978542  [ 6400/60000]
loss: 1.941069  [12800/60000]
loss: 1.934224  [19200/60000]
loss: 1.671774  [25600/60000]
loss: 1.728644  [32000/60000

# References 
https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html